In [ ]:
# default_exp tasks

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export 
from collections import namedtuple
from datetime import datetime, date, time, timedelta
import re
properties = ['task',
              'Estd_Time',
              'Strt_Date',
              'Time_Takn',
              'Comp_Date',
              'Satsfactn',
              'completed']
defaults = ['?']*(len(properties)-1)
defaults[-1] = 0 #default for 'completed' is to be 0
Task = namedtuple('Task', properties, defaults=defaults)

- TODO 
- [ ] Start_Date: today's date or scheduled date of the form 'dd/mm/yy' or 'tomorrow'
- [ ] Time_takn: str of the form '12m'
- [ ] Implement python-dateutil library to better parse dates and support for months, years etc for timedelta

In [ ]:
#export
args_map = {'d':'days',
             'h':'hours',
             'w':'weeks',
             'm':'minutes'}

def str2timeDelta(string):
    
    pattern = re.compile(r"""
    (\d+\.*\d*) #Any real numbers >> Digit decimal digit(optional)
    \s? #To handle space
    ([hmsdw][o]?)  # any of h, m... o is to diff b/w m[in] and m[onth]
    """, re.VERBOSE)                         #remember to add functionality for month later
    args_as_tuples_in_list = re.findall(pattern, string)
    args = {args_map.get(arg_tuple[1]) : float(arg_tuple[0]) 
                for arg_tuple in args_as_tuples_in_list}
    return timedelta(**args)

In [ ]:
#export 
def task_scheduler(s: 'string of task'):
    """
    Asks the user the date on which selected string is to be added to the "today's task list"
    then creates and returns a Task object with Strt_Date field as that date.    
    """
    print(s)
    scheduled_task = Task(s)
    print(f"When should be this on your task list. \n")
    print(f"Enter 'y' to add it on today's list. Today is {date.today()}")
    print(f"You can enter the date in the format of today's date printed above.")
    print(f'Or you can answer in relative terms. Eg: 10w, 5d (only weeks and days supported)')
    date_string = input()
    pattern_for_isodate = re.compile(r'\d{4}-\d{2}-\d{2}')
    
    if 'y' in date_string: 
        scheduled_task = scheduled_task._replace(
            Strt_Date = date.today())  
        
    elif re.match(pattern_for_isodate, date_string): 
        scheduled_task = scheduled_task._replace(
            Strt_Date = date.fromisoformat(date_string))
        
    else: 
        scheduled_task = scheduled_task._replace(
            Strt_Date = date.today() + str2timeDelta(date_string))
        
    return scheduled_task

In [ ]:
#export
def estimated_time(task_tuple):
    print(f'How much time do you think this would take ?')
    t = input("Eg: '45m', '1h'")
    task_tuple = task_tuple._replace( Estd_Time = str2timeDelta(t) )
    return task_tuple

def actual_time(task_tuple):
    print(f'You thought you would take {task_tuple.Estd_Time}. How much time did you actually take ?')
    t = input("Eg: '45m', '1h'")
    task_tuple = task_tuple._replace( Time_Takn = str2timeDelta(t) )
    return task_tuple

def satisfaction(task_tuple):
    print("On a scale of 1-10, how much satisfied do you about the effort you put into this task ?")
    s = float(input())
    task_tuple = task_tuple._replace( Satsfactn = s)
    if task_tuple.completed == 1: 
        task_tuple = task_tuple._replace( CompDate = date.today() ) 
    return task_tuple


In [ ]:
task_scheduler('Check Hacker Rank website') #something is wrong.. check this..

Check Hacker Rank website
When should be this on your task list. 

Enter 'y' to add it on today's list. Today is 2020-06-07
You can enter the date in the format of today's date printed above.
Or you can answer in relative terms. Eg: 10w, 5d (only weeks and days supported)


Task(task='Check Hacker Rank website', Estd_Time='?', Strt_Date=datetime.date(2020, 6, 7), Time_Takn='?', Comp_Date='?', Satsfactn='?', completed=0)

In [ ]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_Tasks.ipynb.
Converted 02_questions.ipynb.
Converted 03_tests.ipynb.
Converted 04_summary image.ipynb.
Converted 11_Tasks.ipynb.
Converted index.ipynb.


In [ ]:
timedelta?

Init signature: timedelta(self, /, *args, **kwargs)
Docstring:     
Difference between two datetime values.

timedelta(days=0, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)

All arguments are optional and default to 0.
Arguments may be integers or floats, and may be positive or negative.
File:           ~/anaconda3/lib/python3.7/datetime.py
Type:           type
Subclasses:     


In [ ]:
date_string = f'{date.today()}'

In [ ]:
date_string.count('-')

2

In [ ]:
names = ['raymond', 'rachel', 'matthew']
colors = ['red', 'green', 'blue','red', 'green', 'red']
d = {}
for color in colors:
    d[color] = d.get(color, 0) + 1


In [ ]:
1000**3

1000000000